In [1]:
import multiprocessing
import os
import time

In [2]:
def do_job(name):
    whoami(name)
    
def whoami(name):
    print(f'Process {os.getpid()} doing {name}')
    
def loopy(name):
    whoami(name)
    start = 1
    stop = 100000
    for i in range(start, stop):
        print(f'Doing {i} of {stop}')
        time.sleep(1)

In [13]:
if __name__ == '__main__':
    whoami('Main program')
    for i in range(4):
        p = multiprocessing.Process(target=do_job, args=(f'loop {i}',))
        p.start()

Process 2919 doing Main program
Process 2945 doing loop 0
Process 2946 doing loop 1
Process 2947 doing loop 2
Process 2948 doing loop 3


In [23]:
if __name__ == '__main__':
    whoami('Main program')
    p = multiprocessing.Process(target=loopy, args=(f'loopy',))
    p.start()  # 背景作業
    print('Backgound processing...')
    time.sleep(5)
    p.terminate()  # 提前終止
    print('Stopped')

Process 2919 doing Main program
Process 2970 doing loopy
Doing 1 of 100000
Backgound processing...
Doing 2 of 100000
Doing 3 of 100000
Doing 4 of 100000
Doing 5 of 100000
Stopped


---

## Queue

In [3]:
import multiprocessing as mp

In [4]:
def washer(dishes, output):
    for dish in dishes:
        print(f'[{time.time() - tic:.2f}] Washing dish ({os.getpid()}): {dish}')
        time.sleep(1)
        output.put(dish)
        
def dryer(input):
    while True:
        dish = input.get()
        print(f'[{time.time() - tic:.2f}] Drying dish ({os.getpid()}): {dish}')
        time.sleep(5)
        input.task_done()

In [5]:
if __name__ == '__main__':
    tic = time.time()
    dish_queue = mp.JoinableQueue()
    dryer_proc1 = mp.Process(target=dryer, args=(dish_queue,))
    dryer_proc2 = mp.Process(target=dryer, args=(dish_queue,))
    dryer_proc1.daemon = True
    dryer_proc2.daemon = True  # 2 dryers
    dryer_proc1.start()
    dryer_proc2.start()

    dishes = ['apple', 'banana', 'orange', 'salad']
    washer(dishes, dish_queue)
    dish_queue.join()

[0.04] Washing dish (3042): apple
[1.04] Drying dish (3047): apple
[1.04] Washing dish (3042): banana
[2.05] Drying dish (3048): banana
[2.05] Washing dish (3042): orange
[3.05] Washing dish (3042): salad
[6.05] Drying dish (3047): orange
[7.06] Drying dish (3048): salad
